# 線形CMDPでの (打ち消しありの) 劣線形リグレット 

参考：
* [Provably Efficient Model-Free Constrained RL with Linear Function Approximation](https://arxiv.org/abs/2206.11889)
* [Towards Achieving Sub-linear Regret and Hard Constraint Violation in Model-free RL](https://proceedings.mlr.press/v238/ghosh24a.html)：こっちのほうが良い結果だけど，わかりやすいので最初の論文についてまとめるよ．

今回は制約付き線形MDPを扱います．

表記：
* $\xi = H / \gamma$で，ここで$\gamma$はSlater変数
* $r$は報酬，$g$はutility．$j=r, g$はどっちでも良いときに使う．
* あとは論文参照．雰囲気でわかるはず．

## アルゴリズム


$$
\begin{aligned}
& \Lambda_h^k \leftarrow \sum_{\tau=1}^{k-1} \phi\left(x_h^\tau, a_h^\tau\right) \phi\left(x_h^\tau, a_h^\tau\right)^T+\lambda \mathbf{I} \\
& w_{r, h}^k \leftarrow\left(\Lambda_h^k\right)^{-1}\left[\sum_{\tau=1}^{k-1} \phi\left(x_h^\tau, a_h^\tau\right)\left[r_h\left(x_h^\tau, a_h^\tau\right)+V_{r, h+1}^k\left(x_{h+1}^\tau\right)\right]\right] \\
& w_{g, h}^k \leftarrow\left(\Lambda_h^k\right)^{-1}\left[\sum_{\tau=1}^{k-1} \phi\left(x_h^\tau, a_h^\tau\right)\left[g_h\left(x_h^\tau, a_h^\tau\right)+V_{g, h+1}^k\left(x_{h+1}^\tau\right)\right]\right] \\
& Q_{r, h}^k(\cdot, \cdot) \leftarrow \min \left\{\left\langle w_{r, h}^k, \phi(\cdot, \cdot)\right\rangle+\beta\left(\phi(\cdot, \cdot)^T\left(\Lambda_h^k\right)^{-1} \phi(\cdot, \cdot)\right)^{1 / 2}, H\right\} \\
& Q_{g, h}^k(\cdot, \cdot) \leftarrow \min \left\{\left\langle w_{g, h}^k, \phi(\cdot, \cdot)\right\rangle+\beta\left(\phi(\cdot, \cdot)^T\left(\Lambda_h^k\right)^{-1} \phi(\cdot, \cdot)\right)^{1 / 2}, H\right\} \\
& \pi_{h, k}^k(a \mid \cdot)=\frac{\exp \left(\alpha\left(Q_{r, h}^k(\cdot, a)+Y_k Q_{g, h}^k(\cdot, a)\right)\right)}{\sum_a \exp \left(\alpha\left(Q_{r, h}^k(\cdot, a)+Y_k Q_{g, h}^k(\cdot, a)\right)\right)} \\
& V_{r, h}^k(\cdot)=\sum_a \pi_{h, k}(a \mid \cdot) Q_{r, h}^k(\cdot, a) \\
& V_{g, h}^k(\cdot)=\sum_a \pi_{h, k}(a \mid \cdot) Q_{g, h}^k(\cdot, a)
\end{aligned}
$$

ここで，$Y$はラグランジュ変数と似たような働きをします．
$$Y_{k+1}=\max \left\{\min \left\{Y_k+\eta\left(b-V_{g, 1}^k\left(x_1\right)\right), \xi\right\}, 0\right\}$$

## リグレットの解析

まず次の性質が成立することを念頭に置いておきます（証明は略．コーシー・シュワルツとClippingからすぐ出ると思う）：
* $\left\|w_{j, h}^\pi\right\| \leq 2 H \sqrt{d}$
* $\left\|w_{j, h}^k\right\| \leq 2 H \sqrt{d k / \lambda}$
* （勾配法の解析から出るはず）：任意の$Y \in [0, \xi]$で，$\sum_{k=1}^K\left(Y-Y_k\right)\left(b-V_{g, 1}^k\left(x_1\right)\right) \leq \frac{Y^2}{2 \eta}+\frac{\eta H^2 K}{2}$


---

**リグレットの分解**

次が成立することを利用してリグレットを分解します：任意の$Y \in [0, \xi]$について，

$$
\begin{aligned}
& \underbrace{\sum_{k=1}^K\left(V_{r, 1}^{\pi^*}\left(x_1\right)-V_{r, 1}^{\pi_k}\left(x_1\right)\right)}_{報酬のリグレット}+Y \underbrace{\sum_{k=1}^K\left(b-V_{g, 1}^{\pi_k}\left(x_1\right)\right)}_{違反の（弱）リグレット} \leq \frac{1}{2 \eta} Y^2+\frac{\eta}{2} H^2 K+ \\
& \underbrace{\sum_{k=1}^K\left(V_{r, 1}^{\pi^*}\left(x_1\right)+Y_k V_{g, 1}^{\pi^*}\left(x_1\right)\right)-\left(V_{r, 1}^k\left(x_1\right)+Y_k V_{g, 1}^k\left(x_1\right)\right)}_{\mathcal{T}_1}+ \\
& \underbrace{\sum_{k=1}^K\left(V_{r, 1}^k\left(x_1\right)-V_{r, 1}^{\pi_k}\left(x_1\right)\right)+Y \sum_{k=1}^K\left(V_{g, 1}^k\left(x_1\right)-V_{g, 1}^{\pi_k}\left(x_1\right)\right)}_{\mathcal{T}_2}
\end{aligned}
$$

**証明**：

$$
\begin{aligned}
Y \sum_{k=1}^K\left(b-V_{g, 1}^{\pi_k}\left(x_1\right)\right) & =\sum_k\left(Y-Y_k\right)\left(b-V_{g, 1}^k\left(x_1\right)\right)+Y_k\left(b-V_{g, 1}^k\right)+Y\left(V_{g, 1}^k\left(x_1\right)-V_{g, 1}^{\pi_k}\left(x_1\right)\right) \\
& \leq \frac{1}{2 \eta} Y^2+\frac{\eta}{2} H^2 K+\sum_{k=1}^K\left(Y_k b-Y_k V_{g, 1}^k\left(x_1\right)\right)+Y\left(V_{g, 1}^k\left(x_1\right)-V_{g, 1}^{\pi_k}\left(x_1\right)\right) \\
& \leq \frac{1}{2 \eta} Y^2+\frac{\eta}{2} H^2 K+\sum_{k=1}^K\left(Y_k V_{g, 1}^{\pi^*}\left(x_1\right)-Y_k V_{g, 1}^k\left(x_1\right)\right)+\sum_{k=1}^K \left(YV_{g, 1}^k\left(x_1\right)-Y V_{g, 1}^{\pi_k}\left(x_1\right)\right)
\end{aligned}
$$

あとは入れ替えたら終わりです．

---



$Y=0$にしてみると，報酬についてのリグレットがバウンドできることがわかります．よって，$\mathcal{T}_{1, 2}$をバウンドできれば良さそうです．
早速$\mathcal{T}$のバウンドを出していきましょう．

## $\mathcal{T}$のバウンド

まず，次が成立します：

$$
\| \sum_{\tau=1}^{k-1} \phi_{j, h}^\tau\left[V_{j, h+1}^k\left(x_{h+1}^\tau\right)-\mathbb{P}_h V_{j, h+1}^k\left(x_h^\tau, a_h^\tau\right) \|_{\left(\Lambda_h^k\right)^{-1}}\right.
$$

* コメント：これは[RL_General_linearMDP-Model-based.ipynb](RL_General_linearMDP-Model-based.ipynb) と一見異なるが，実は全く同じ．Model-basedの方の記法をよく確認すれば，一緒．

**証明**

* [Provably Efficient Exploration in Policy Optimization](https://arxiv.org/abs/1912.05830)これで行ける気がするかもしれないが，この論文を使った方法はダメ．参考論文では$w_h^k \leftarrow\left(\Lambda_h^k\right)^{-1} \sum_{\tau=1}^{k-1} \phi_h^\tau\left(x_h^\tau, a_h^\tau\right) \cdot V_{h+1}^\tau\left(x_{h+1}^\tau\right)$の形で更新していて，過去の価値関数を全部足して場合はこの論文が使え，Coveringが回避できるが，上の価値更新では$V^k$についての話なので，使えない．


次のQ, V関数のクラスを考えます．

* $\mathcal{Q}_j=\left\{Q \mid Q(\cdot, \cdot)=\min \left\{w_j^T \phi(\cdot, \cdot)+\beta \sqrt{\phi^T(\cdot, \cdot)^T \Lambda^{-1} \phi(\cdot, \cdot)}, H\right\}\right\}$
* $\mathcal{V}_j=\left\{V_j \mid V_{r, g}(\cdot)=\sum_a \pi_{r, g, Y}(a \mid \cdot) Q_{j}(\cdot, a) ; Q_{r} \in \mathcal{Q}_{r}, Q_{g} \in \mathcal{Q}_{g}, Y \in[0, \xi]\right\}$
    * ここで，$\pi_{r, g, Y}(a \mid \cdot)=\operatorname{SofT-MAX}{ }_\alpha^a\left(Q_r(\cdot, \cdot)+Y Q_g(\cdot, \cdot)\right)$
    * つまり，複合Q関数が与えられたときに，そのSoftmaxによって作られるV関数です．


このクラスについて，次が成立します：

アルゴリズム中の任意の$V_j$について，
$\left(\tilde{w}_r, \tilde{w}_g, \tilde{\beta}, \Lambda, \tilde{Y}\right)$によってパラメータ化される，次を満たす$\tilde{V}_j \in \mathcal{V}_j$が存在する：
$$
\operatorname{DIST}\left(V_j, \tilde{V}_j\right) \leq \epsilon
$$
ここで，$\operatorname{DIST}\left(V_j, \tilde{V}_j\right)=\sup _x\left|V_j(x)-\tilde{V}_r(x)\right|$.

また，$\mathcal{V}_j$の$\epsilon$被覆について，
$$
\log N_\epsilon^{V_j} \leq d \log \left(1+8 H \frac{\sqrt{d k}}{\sqrt{\lambda} \epsilon^{\prime}}\right)+d^2 \log \left[1+8 d^{1 / 2} \beta^2 /\left(\lambda\left(\epsilon^{\prime}\right)^2\right)\right]+\log \left(1+\frac{\xi}{\epsilon^{\prime}}\right)
$$
が成り立つ．ここで，$\epsilon^{\prime}=\frac{\epsilon}{H 2 \alpha(1+\xi+H)+1}$．

**証明**

まず，$Q_j$についての$\epsilon$被覆を導出する必要がありますが，これは[RL_General_linearMDP](RL_General_linearMDP-Model-based.ipynb)とほとんど同じなので省略します．$\left\{w \in \mathbb{R}^d\|\mid w\| \leq 2 H \sqrt{d k / \lambda}\right\}$と$\left\{\mathbf{A} \in \mathbb{R}^{d \times d}\| \| \mathbf{A} \|_F \leq d^{1 / 2} \beta^2 \lambda^{-1}\right\}$について，次が成り立ちます：
$$
\log N_\epsilon^{Q_j} \leq d \log \left(1+\frac{8 H \sqrt{d k}}{\sqrt{\lambda} \epsilon}\right)+d^2 \log \left[1+8 d^{1 / 2} \beta^2 /(\lambda \epsilon)^2\right]
$$

---

また，当たり前ですが，$\epsilon'$だけ近い$\tilde{Q}_j$があれば，複合Q関数も十分近くなることが言えます．一応まとめておきましょう．

もし
* $\operatorname{dist}\left(Q_r^k, \tilde{Q}_r\right) \leq \epsilon^{\prime}$,
* $\operatorname{dist}\left(Q_g^k, \tilde{Q}_g\right) \leq \epsilon^{\prime}$, 
* $\left|\tilde{Y}_k-Y_k\right| \leq \epsilon^{\prime}$, ならば，

$$\operatorname{dist}\left(Q_r^k+Y_k Q_o^k, \tilde{Q}_r+\tilde{Y}_k \tilde{Q}_g\right) \leq \epsilon^{\prime}(1+\xi+H)$$

証明は略（簡単なので）

---

さらに，$\epsilon'$だけ近い$\tilde{Q}_j$と$\tilde{Y}$があれば，$\pi$と$\tilde{\pi}$が近くなることも言えます．

* $\operatorname{dist}\left(Q_r^k, \tilde{Q}_r\right) \leq \epsilon^{\prime}$,
* $\operatorname{dist}\left(Q_g^k, \tilde{Q}_g\right) \leq \epsilon^{\prime}$, 
* $\left|\tilde{Y}_k-Y_k\right| \leq \epsilon^{\prime}$,
* $\pi(a \mid \cdot)=\operatorname{Soft}-\operatorname{MAx}_\alpha^a\left(\left(Q_r(\cdot, \cdot)+Y_k Q_g(\cdot, \cdot)\right)\right.$
* $\tilde{\pi}(a \mid \cdot)=\operatorname{SOFT-MAX}{ }_\alpha^a\left(\left(\tilde{Q}_r(\cdot, \cdot)+\tilde{Y}_k \tilde{Q}_g(\cdot, \cdot)\right)\right.$ならば，

$$
\|\pi(\cdot \mid x)-\tilde{\pi}(\cdot \mid x)\|_1 \leq 2 \alpha \epsilon^{\prime}(1+\xi+H)
$$

が言えます．つまり，$\alpha$を小さくする（貪欲方策から離れる）か，$\epsilon'$を小さくする（coveringを小さくする）ことで方策同士の差を抑えることができます．あとは簡単なので省略．


## TODO: 続き書く（あとは多分Elliptical potential lemmaとかでいける）